## Zhihu

**知乎爬虫主要通过api调用获取数据信息,关键词搜索数据返回api暂时在维修中无法使用，其通过Web Scraper插件使用实现。**

In [ ]:
import requests
import json
import datetime
import time
from selectolax.parser import HTMLParser
import pandas as pd
import re

In [ ]:
def download_answer(question_id, start):
#    基于问题id获取单个问题下所有回答数据信息
    print(f"问题{question_id},第{start}条。")
    url = "https://www.zhihu.com/api/v4//questions/{}/answers".format(question_id)
    print(url)
    headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36',
               'referer':'https://www.zhihu.com/question/{}'.format(question_id),
              } 
    
    para = {
    'include':'data[*].is_normal,admin_closed_comment, \
    reward_info,is_collapsed,annotation_action,annotation_detail,\
    collapse_reason,is_sticky,collapsed_by,suggest_edit,comment_count,\
    can_comment,content,editable_content,attachment,voteup_count,\
    reshipment_settings,comment_permission,created_time,\
    updated_time,review_info,relevant_info,question,excerpt,is_labeled,\
    paid_info,paid_info_content,relationship.is_authorized,is_author,voting,is_thanked,\
    is_nothelp,is_recognized;data[*].mark_infos[*].url;data[*].author.follower_count,\
    vip_info,badge[*].topics;data[*].settings.table_of_content.enabled',
    'offset':start,
    'limit':20,
    'sort_by':'default',
    'platform':'desktop'
    }
    
    global df   

    response = requests.get(url,params=para,headers = headers)
    response.encoding = response.apparent_encoding
    if response.status_code == 200:
        res = json.loads(response.text)
        if res['data']:
            for answer in res['data']:
                author = answer['author']['name']
                fans = answer['author']['follower_count'] 
                content = HTMLParser(answer['content']).text()
                #content = answer['content']
                created_time = datetime.datetime.fromtimestamp(answer['created_time'])
                updated_time = datetime.datetime.fromtimestamp(answer['updated_time'])
                comment = answer['comment_count']
                voteup = answer['voteup_count']
                link = answer['url']
                row = {
                    'author':[author],
                    'fans_count':[fans],
                    'content':[content],
                    'created_time':[created_time],
                    'updated_time':[updated_time],
                    'comment_count':[comment],
                    'voteup_count':[voteup],
                    'url':[link]
                }
                df = df.append(pd.DataFrame(row),ignore_index=True)
            if len(res['data'])==20: 
                download_answer(question_id,start+20)
                time.sleep(1)
        else:
            return None
    else:
        return None

In [ ]:
urls_list = list(pd.read_csv(r"C:\Users\Ljinjin\Desktop\data\zhihu\zhihu_urls.csv").iloc[:,4])
#Web Scraper插件也可获取问题id

# 知乎API维护中...
# def download_question(risk_keyword, start):
# #     基于风险关键词获取问题id
#     url = 'https://www.zhihu.com/api/v4/search_v3?'
#     headers = {
#         'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36',
#     }
#     para = {
#         'gk_version':'gz-gaokao',
#         't':'general',
#         'q':'人工智能 {}'.format(risk_keyword),
#         'correction':1,
#         'offset':start,
#         'limit':20,
#         'filter_fields':'',
#         'lc_idx':0,
#         'show_all_topics':0,
#         'search_source':'Normal'
#     }    
#     response = requests.get(url,params=para,headers = headers)
#     response.encoding = response.apparent_encoding
#     if response.status_code == 200:
#         print(response.text)
#         return response.text
#     else:
#         return None

In [ ]:
questionId = []
def getQuestionId(href):
    res = re.findall(r'https://www.zhihu.com/question/([0-9]+)',href)
    if(len(res)!=0):
        questionId.append(res[0])
for url in urls_list:
    getQuestionId(url)

In [ ]:
df = pd.DataFrame(columns=('author','fans_count','content','created_time','updated_time','comment_count','voteup_count','url'))

for id in questionId:
    try:
        download_answer(id,0)
    except:
        continue

In [ ]:
df.to_csv(r"./zhihu.csv", encoding='utf-8-sig')

## Jianshu

**基于Selenium+直接页面解析获取数据的爬虫实现。**  
此方式集中爬取消耗时间过长，推荐**分批次**爬取数据。

In [ ]:
from selenium import webdriver
import pandas as pd
import time

from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def generate_urls():

    list = driver.find_elements_by_css_selector("a.title")
    linkset = set()
    for item in list:
        try :
            link = item.get_attribute("href")
            linkset.add(link)
        except:
            continue
    print(linkset)
    return linkset

def generate_comments_one_page(url):
        # title author time content url num
        import time

        driver.get(url)

        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1._1RuRku")))
        except:
            print("access failed")
            return


        try:
            title = driver.find_element_by_css_selector("h1._1RuRku").text
        except:
            title = ""
            print("title missing")

        try :
            author = driver.find_element_by_css_selector(".FxYr8x a").text
        except:
            author = ""
            print("author missing")

        try:
            time = driver.find_element_by_css_selector("time").text
        except:
            time = ""
            print("time missing")

        try:
            content = driver.find_element_by_css_selector("article").text
        except:
            content = ""
            print("content missing")

        try:
            support = driver.find_element_by_css_selector("span._1LOh_5").text
        except:
            support = ""
            print("support missing")


        comment = [title, author, time, content, url, support]
        return comment


def write_csv(datalist, path):
    headers=["title","author","time","content","url","num"]
    test = pd.DataFrame(columns=headers,data=datalist)
    test.to_csv(path, encoding='utf-8-sig')
    return test

def one_time_crawl(url):  # 利用set去重
    # 访问搜索页

    for i in range(0,101):
        cur_page = url.replace(r"page=1","page="+str(i))
        driver.get(cur_page)
        # 等待页面加载成功
        refresh()
        # 最近三天 按钮
        driver.find_element_by_css_selector("svg").click()
        three_day_element = driver.find_element_by_css_selector("li.v-select-options-item:nth-of-type(4)")
        ActionChains(driver).move_to_element(three_day_element).click()
        # 读取当前页文章url
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a.title")))
        for item in generate_urls():
            urls.add(item)
        print(len(urls))

        time.sleep(0.5)
    write_url()
    return


def refresh():
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.title")))
    except:
        driver.refresh()
        refresh()


def write_url():
    header2 = ["urls"]
    test = pd.DataFrame(columns=header2,data=urls)
    test.to_csv(r"./jianshu_urls.csv", encoding='utf-8-sig')

def input_url():
    urls={
    "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E9%A3%8E%E9%99%A9&page=1&type=note" #人工智能 风险
    "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%A8%81%E8%83%81&page=1&type=note" #人工智能 威胁
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%AE%89%E5%85%A8%E9%9A%90%E6%82%A3&page=1&type=note" #人工智能 安全隐患
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E9%99%A9&page=1&type=note" #人工智能 危险
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E6%9C%BA&page=1&type=note" #人工智能 危机
    "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%90%8E%E6%9E%9C&page=1&type=note" #人工智能 后果
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E5%AE%B3&page=1&type=note" #人工智能 危害
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E9%99%A9&page=1&type=note" #人工智能 安全事件
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E9%A2%84%E9%98%B2&page=1&type=note" #人工智能 预防
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%AE%89%E5%85%A8%E4%BA%8B%E6%95%85&page=1&type=note" #人工智能 安全事故
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E9%99%A9&page=1&type=note" #人工智能 风险管理
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E8%B4%9F%E9%9D%A2%E5%BD%B1%E5%93%8D&page=1&type=note" #人工智能 负面影响
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E7%81%BE%E5%AE%B3&page=1&type=note" #人工智能 灾害
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%9B%B0%E5%A2%83&page=1&type=note" #人工智能 困境  
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E6%AC%BA%E8%AF%88&page=1&type=note" #人工智能 欺诈
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E6%AC%BA%E8%AF%88&page=1&type=note" #人工智能 失误
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E6%BC%8F%E6%B4%9E&page=1&type=note" #人工智能 漏洞
    }
    return urls
#     try:
#         data = pd.read_csv(r"./data/jianshu_urls.csv",encoding= 'utf-8-sig')
#         return set(data["urls"].tolist())

#     except:
#         print("url file not exist")

urls = input_url()
driver = webdriver.Chrome(r'C:\Users\Ljinjin\AppData\Local\Google\Chrome\chromedriver.exe')
comments = []
urls_to_scrape = [#"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E9%A3%8E%E9%99%A9&page=1&type=note" #人工智能 风险
     "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%A8%81%E8%83%81&page=1&type=note" #人工智能 威胁
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%AE%89%E5%85%A8%E9%9A%90%E6%82%A3&page=1&type=note" #人工智能 安全隐患
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E9%99%A9&page=1&type=note" #人工智能 危险
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E6%9C%BA&page=1&type=note" #人工智能 危机     
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%90%8E%E6%9E%9C&page=1&type=note" #人工智能 后果
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E5%AE%B3&page=1&type=note" #人工智能 危害
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E9%99%A9&page=1&type=note" #人工智能 安全事件
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E9%A2%84%E9%98%B2&page=1&type=note" #人工智能 预防
    ,"https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%AE%89%E5%85%A8%E4%BA%8B%E6%95%85&page=1&type=note" #人工智能 安全事故
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%8D%B1%E9%99%A9&page=1&type=note" #人工智能 风险管理
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E8%B4%9F%E9%9D%A2%E5%BD%B1%E5%93%8D&page=1&type=note" #人工智能 负面影响
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E7%81%BE%E5%AE%B3&page=1&type=note" #人工智能 灾害
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E5%9B%B0%E5%A2%83&page=1&type=note" #人工智能 困境  
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E6%AC%BA%E8%AF%88&page=1&type=note" #人工智能 欺诈
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E6%AC%BA%E8%AF%88&page=1&type=note" #人工智能 失误
    , "https://www.jianshu.com/search?q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD%20%E6%BC%8F%E6%B4%9E&page=1&type=note" #人工智能 漏洞
]


while (len(str(urls)) < 7500):
    for url_to_scrape in urls_to_scrape:
        one_time_crawl(url_to_scrape)

for url in urls:
    print(url)
    this_comment = generate_comments_one_page(url)
    if this_comment is not None:
        comments.append(this_comment)
    print(len(comments))
    time.sleep(0.3)




path = r'./jianshu.csv' 
write_csv(comments, path)
driver.quit()

## Douban

**基于Selenium+直接页面解析获取数据的爬虫实现。**  
此方式集中爬取消耗时间过长，推荐**分批次**爬取数据；此外模拟浏览器的豆瓣首页弹出时可通过自身**账号登录**以保证后续爬取数据的顺利。

In [ ]:
from selenium import webdriver
import time, datetime
from bs4 import BeautifulSoup
import pandas as pd
import re

driver = webdriver.Chrome(r'C:\Users\Ljinjin\AppData\Local\Google\Chrome\chromedriver.exe')  #模拟器存放文件夹的位置
driver.get('https://www.douban.com/')
time.sleep(20)


def generate_urls():
    """
    获取人工智能搜索页的豆瓣日记信息
    """
    base_urls = [
           "https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD"  #人工智能, 
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E9%A3%8E%E9%99%A9"  #人工智能 风险
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%A8%81%E8%83%81" #人工智能+威胁
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%AE%89%E5%85%A8%E9%9A%90%E6%82%A3" #安全隐患
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%8D%B1%E9%99%A9" #危险
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%8D%B1%E6%9C%BA" #危机
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%90%8E%E6%9E%9C" #后果 2
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%8D%B1%E5%AE%B3" #危害 2
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%8D%B1%E9%99%A9" #危险 2
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E9%A2%84%E9%98%B2" #预防 3
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%AE%89%E5%85%A8%E4%BA%8B%E6%95%85" #安全事故 3
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E8%B4%9F%E9%9D%A2%E5%BD%B1%E5%93%8D" #负面影响 3
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E7%81%BE%E5%AE%B3" #灾害
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E5%9B%B0%E5%A2%83" #困境
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E6%AC%BA%E8%AF%88" #欺诈
        ,"https://www.douban.com/search?cat=1015&q=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD+%E6%BC%8F%E6%B4%9E" #漏洞
        ] 
    
    urls = []

    for base_url in base_urls:
        urls.append(base_url)
        prefix = "https://www.douban.com/j/search?q="
        suffix = "&start={param}&cat=1015"
        middle = base_url[41:]
        for p in range(1, 101):
            url = prefix + middle + suffix.format(param = p * 20)
            urls.append(url)

    return urls


def generate_hrefs(urls):

    hrefs = []
    for url in urls:
        driver.get(url)
        search_window = driver.current_window_handle
        pageContent = driver.page_source
        time.sleep(4)

        pattern = re.compile('sid: (\d+)', re.S)
        results = re.findall(pattern, pageContent)

        for line in results:
            href = "https://www.douban.com/note/" + line
            hrefs.append(href)

    return hrefs

#设置爬取的文章的时间段
# this_year = time.strptime("2020-08-08 00:00:00","%Y-%m-%d %H:%M:%S")
# this_year = "2021-11-06 08:30:46"


def generate_articles(hrefs):
    
    print(hrefs)

    articles = []
    for href in hrefs:
        print(href)

        driver.get(href)

        search_window = driver.current_window_handle
        pageSource = driver.page_source
        time.sleep(1)

        try :
            soup = BeautifulSoup(pageSource)
            title_author = soup.find("div", attrs={"class": "note-header note-header-container"})
            # 文章标题
            title = title_author.find("h1", attrs={"": ""}).text
            # 文章作者
            author = title_author.find("a", attrs={"class": "note-author"}).text
            # 文章发布时间
            date = title_author.find("span", attrs={"class": "pub-date"}).text
#             publish_time = time.strptime(date, "%Y-%m-%d %H:%M:%S")
#             print(date)
#             if date < this_year:
#                 continue

        # 文章正文
            res = soup.find("div", attrs={"id": "link-report"})
            note = res.find("div", attrs={"class": "note"})
            txt = note.text

        except :
            continue


        # 点赞数
        try:
            support = soup.find("span", attrs={"class": "react-num"}).text

        except:
            support = ""
        # 标签
        try:
            topics = soup.find('div', attrs={"class": "mod-tags"})
            topics_a = topics.find_all("a")

            tags = []
            for topic_a in topics_a:
                topic = topic_a.text
                tags.append(topic)

            tag = " ".join(tags)

        except:
            tag = ""

        # 评论信息
        try:
            divs = soup.find_all("div", attrs={"class": "comment-content"})
            meta_headers = soup.find_all("div", attrs={"class": "meta-header"})

            comments = []
            for line1, line2 in zip(divs, meta_headers):
                span = line1.find("span")
                review = span.text
                a = line2.find_all("a", attrs={"title": re.compile(".*")})
                comment_user = a[0].text
                comment_time = line2.find("time").text
                comment = [comment_user, review, comment_time]
                comments.append(comment)

        except:
            comments = []

        article = [href, title, author, date, txt, support, tag, comments]
        print(article)
        articles.append(article)
    
    print(articles)
    return articles

def write_csv(datalist, path):
    headers=["link","title","author","date","text","vote","label","comment"]
    test = pd.DataFrame(columns=headers,data=datalist)
    test.to_csv(path, encoding='utf-8-sig')
    return test


urls = generate_urls()
hrefs = generate_hrefs(urls)
articles = generate_articles(hrefs)
driver.quit()
path = r'./douban.csv'

write_csv(articles, path)

In [ ]:
douban.to_csv('douban.csv', encoding='utf-8-sig')